# Extracción de datos Datasets Google Maps y YELP


### Importaciones


In [1]:
import pandas as pd
import numpy as np
import json
import datetime
import os
import pyarrow.parquet as pq
from datetime import datetime

### Utiles


In [2]:
state_abreviations = [
    "AL",
    "AK",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DE",
    "FL",
    "GA",
    "HI",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY",
]

state_dictionary = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming",
}

In [3]:
generated_dir = "Generated"

os.mkdir(generated_dir)
os.chdir(generated_dir)
os.mkdir("Google")
os.mkdir("Yelp")
os.chdir("../")

os.getcwd()

FileExistsError: [WinError 183] No se puede crear un archivo que ya existe: 'Generated'

# 1. Extracción con los datasets de Google Maps


## 1.1 Metada de Sitios


Recorremos todo el directorio, lo hacemos archivo por archivo y línea por línea ya que no se puede abrir directamente los archivos por su dimensión y porque no están en formato de array, sino están constituidos en un registro por lìnea.
Durante la lectura filtramos los que incluyan <code>Restaurant</code> en el campo de categoría, para alivianar el dataset final.


In [ ]:
# Tiempo de Demora Medio: 51 segundos.
lineas_json = []

# Son 11 archivos con un ordinal, del 1 al 11
for i in range(1, 12):
    path = f"Datasets/Google Maps/metadata-sitios/{i}.json"
    with open(path, "r") as file:
        for l in file:
            try:
                linea_j = json.loads(l)
                if "restaurant" in " ".join(linea_j["category"]).lower():
                    lineas_json.append(linea_j)
            except:
                pass

df = pd.DataFrame(lineas_json)

df.head(3)

In [ ]:
df.loc[5, "address"]

Exportamos a formato Parquet


In [ ]:
df.to_parquet(r"Generated\Google\metada_sitios.parquet")

Tamaño Directorio <code>metadata-sitios</code>: 2.76 Gb

Tamaño Archivo <code>metada_sitios.parquet</code>: 60.43 Mb


In [ ]:
df.info()

Dimensiones: 212.014 filas x 15 Columnas


## 1.1.1 Obtención de información de Estados


En base al campo <code>Address</code> obtenemos el estado donde se encuentra el negocio. Nos servirá para luego seleccionar los estados con más restaurantes.


In [ ]:
def get_state_ab(st):
    try:
        state = st.split(", ")[-1].split(" ")[0]
        if state in state_abreviations:
            return state
        else:
            return np.nan
    except:
        return np.nan


df["state_ab"] = df["address"].apply(get_state_ab)

De esta manera conseguimos el top 5 de los estados con más restaurantes


In [ ]:
top_5 = df["state_ab"].value_counts().head(5).index.to_list()

In [ ]:
top_5

Completamos el campo estado que es más descriptivo


In [ ]:
df["us_state"] = df["state_ab"].map(state_dictionary)

In [ ]:
df["us_state"].head(5)

Obtenemos un arreglo de URLs de los archivos correspondientes para cada estado del top 5, con el fin de extraer los datos en un bucle.


In [ ]:
top_5_url = [
    f"Datasets/Google Maps/reviews-estados/review-{state_dictionary[i].replace(' ', '_')}/"
    for i in top_5
]



top_5_url

Creamos un diccionario con la cantidad de archivos por cada directorio de estados, con el fin de utilizarlo en un bucle en la extracción de datos.


In [ ]:
cantidad_archivos = {}

for i in top_5_url:
    for j in os.walk(i):
        cantidad_archivos[i] = len(j[2])

cantidad_archivos

Comprobación


In [ ]:
for i in os.walk("Datasets/Google Maps/reviews-estados/review-Pennsylvania"):
    print(len(i[2]))

In [ ]:
cantidad_archivos["Datasets/Google Maps/reviews-estados/review-Pennsylvania/"]

## 1.2 Reviews Estados


Ya con los estados elegidos estamos en condiciones de ingestar los datos de las carpetas correspondientes dentro del directorio <code>reviews-estados</code>.
Es información masiva lo que genera un archivo de grandes dimensiones, sin embargo previamente filtramos por el parámetro de año <code>2017-2019</code> valiéndonos del campo <code>time</code>, que tiene es un <code>timestamp</code>, pero con 3 digitos más que el usado por <code>datetime</code> de Python. Le agregamos el campo <code>Estado</code> que es más descriptivo.


In [ ]:
### Demora 7 minutos y 40 segundos, 11 minutos, varía

lineas_json_revs_google = []

for i in top_5_url:
    count = 0
    for c in range(1, cantidad_archivos[i] + 1):
        with open(str(i) + str(c) + ".json", "r", encoding="utf-8") as f:
            for s in f:
                linea = json.loads(s)
                linea["anio"] = datetime.datetime.fromtimestamp(
                    linea["time"] / 1000
                ).year
                linea["estado"] = i.split("-")[-1][:-1]

                if linea["anio"] in [2017, 2018, 2019]:
                    lineas_json_revs_google.append(linea)

df_revs_google = pd.DataFrame(lineas_json_revs_google)

df_revs_google.head(3)

In [ ]:
merge_site_reviews = pd.merge(df_revs_google, df, left_on="gmap_id", right_on="gmap_id")

In [ ]:
merge_site_reviews

In [ ]:
merge_site_reviews.to_parquet(r"Generated\Google\merge_site_reviews.parquet")

In [ ]:
df_revs_google.to_parquet(r"Generated\Google\reviews-estados.parquet")

Tamaño archivo: 760 Mb

Tamaño dataset: 24.3 Gb


In [ ]:
df_revs_google.info()

Tamaño 8.339.179 filas x 10 Columnas.


# 2. Extracción de los Dataset de YELP


### 2.1 Business


Contiene los datos de las entidades negocios de Yelp, a un primer vistazo tiene las columnas duplicadas, por lo que hay que hacer un recorte, ya que la segunda mitad tiene datos vacíos en su inmensa mayoría.


In [ ]:
url_business = r"Datasets\Yelp\business.pkl"

df_business = pd.read_pickle(url_business)

df_business = df_business.iloc[:, :-14]

In [ ]:
df_business.sample(3)

In [ ]:
df_business.shape

Luego con la ayuda del campo <code>state</code> filtramos los negocios que se encuentran en los estados seleccionados en nuestro análisis.


In [ ]:
df_business = df_business[df_business.state.isin(top_5)]

In [ ]:
df_business.info()

Seguimos filtrando a través del campo <code>categories</code>, para obtener los negocios que son restaurantes.


In [ ]:
def is_restaurant(st):
    try:
        test = "".join(st).lower()
        return "restaurant" in test
    except:
        return False


df_business = df_business[df_business["categories"].apply(is_restaurant)]

In [ ]:
df_business.info()

In [ ]:
df_business.to_parquet(r"Generated\Yelp\bussines.parquet")

### 2.2 Checkin


In [ ]:
lineas_json = []
path_checkin = r"Datasets\Yelp\checkin.json"
with open(path_checkin, "r", encoding="utf-8") as file:
    for l in file:
        try:
            linea_j = json.loads(l)
            anio = linea_j["date"][:4]
            # if 'restaurant' in " ".join(linea_j['category']).lower():
            if anio in ["2017", "2018", "2019"]:
                lineas_json.append(linea_j)
        except:
            pass

df_checkin = pd.DataFrame(lineas_json)

In [ ]:
df_checkin

In [ ]:
merge_business_checkin = pd.merge(
    df_business, df_checkin, left_on="business_id", right_on="business_id"
)

In [ ]:
merge_business_checkin.info()

In [ ]:
df_checkin.to_parquet(r"Generated\Yelp\checkin.parquet")
merge_business_checkin.to_parquet(r"Generated\YELP\business_checkin.parquet")

### 2.3 Tips


Realizamos la extracción de los datos y filtramos por año según nuestro análisis.


In [ ]:
lineas_json = []
path_tip = r"Datasets\Yelp\tip.json"
with open(path_tip, "r", encoding="utf-8") as file:
    for l in file:
        try:
            linea_j = json.loads(l)
            anio = linea_j["date"][:4]
            if anio in ["2017", "2018", "2019"]:
                lineas_json.append(linea_j)
        except:
            pass

df_tip = pd.DataFrame(lineas_json)

In [ ]:
df_tip.sample(5)

In [ ]:
df_tip.to_parquet(r"Generated\Yelp\tip.parquet")

Unimos el el dataframe de tips con el de negocios


In [ ]:
tips_merged = pd.merge(
    df_tip, df_business, left_on="business_id", right_on="business_id"
)

In [ ]:
tips_merged.sort_values("business_id").head(3)

In [ ]:
tips_merged.to_parquet(r"Generated\Yelp\business_tip.parquet")

### 2.4 Review


In [ ]:
df_reviews_url = r"Datasets\Yelp\review.json"

Usamos el mismo método de linea por linea, y en el proceso filtramos por año y por las reseñas que han sido votadas como útiles.


In [ ]:
# 44 segundos

lineas_json_review = []

with open(df_reviews_url, "r", encoding="utf-8") as f:
    count = 0
    for i in f:
        linea = json.loads(i)
        anio = linea["date"][:4]
        if anio in ["2017", "2018", "2019"] and linea["useful"] == 1:
            lineas_json_review.append(linea)


df_reviews = pd.DataFrame(lineas_json_review)

In [ ]:
df_reviews.sample(10)

In [ ]:
df_reviews.info()

Aligeramos el dataset con unos downgrades de tipo de variables.


In [ ]:
df_reviews["funny"] = df_reviews["funny"].astype("int8")
df_reviews["stars"] = df_reviews["stars"].astype("int8")
df_reviews["cool"] = df_reviews["cool"].astype("int8")

df_reviews.drop("useful", axis=1, inplace=True, errors="ignore")

In [ ]:
df_reviews.to_parquet(r"Generated\Yelp\review.parquet")

### 2.4 Users Yelp


In [ ]:
parquet_file = pq.ParquetFile(r"Datasets\Yelp\user.parquet")

arr_df = []

for batch in parquet_file.iter_batches():
    # count = count +1
    batch_df = batch.to_pandas()
    batch_df["elite"] = batch_df["elite"].apply(lambda x: x.split(","))
    batch_df["elite_len"] = batch_df["elite"].apply(lambda x: len(x))
    batch_df = batch_df.query("elite_len > 1")
    arr_df.append(batch_df)

df_users = pd.concat(arr_df)

In [ ]:
df_users.sample(5)

In [ ]:
df_users.info()

In [ ]:
df_users.reset_index(inplace=True)
df_users.drop("index", axis=1, inplace=True, errors="ignore")

In [ ]:
df_users.to_parquet(r"Generated\Yelp\users_extracted.parquet")

# NORMALIZACIÓN DE DATOS


### **GOOGLE**

#### _1. MERGE_SITE_REVIEWS_


In [4]:
df_maps_restaurantes = pd.read_parquet(r'Generated\Google\metada_sitios.parquet')
df_maps_reviews = pd.read_parquet(r'Generated\Google\merge_site_reviews.parquet')
df_yelp_restaurantes = pd.read_parquet(r'Generated\Yelp\bussines.parquet')
df_yelp_checkin = pd.read_parquet(r'Generated\YELP\business_checkin.parquet')
df_yelp_tips = pd.read_parquet(r'Generated\Yelp\business_tip.parquet')
df_yelp_reviews = pd.read_parquet(r'Generated\Yelp\review.parquet')
df_yelp_users = pd.read_parquet(r'Generated\Yelp\users_extracted.parquet')

In [5]:
df_maps_reviews.sample(3)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,state_ab,us_state
1115485,113947259774124030025,Vasilios Pilios,1516727309211,5,Great,None,None,0x89c25f233bbca7ab:0x9951b1c25edf8626,2018,New_York,...,4.3,399,₩,"[[Sunday, 10AM–10PM], [Monday, 10AM–10PM], [Tu...",{'Accessibility': ['Wheelchair-accessible entr...,Open ⋅ Closes 10PM,"[0x89c25f4860490013:0xc149667847aa264b, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...,None,None
501350,109512706805991449403,Amber Crabtree,1500251798678,5,Here stopping through Salinas. The cashier was...,None,None,0x808dff292d1e280f:0xa4b85e878e768610,2017,California,...,3.3,958,₩,"[[Monday, 9AM–12AM], [Tuesday, 9AM–12AM], [Wed...",{'Accessibility': ['Wheelchair-accessible car ...,Open ⋅ Closes 12AM,"[0x808dff240a1afaa1:0x8a23a16fb251a337, 0x808d...",https://www.google.com/maps/place//data=!4m2!3...,None,None
441714,107783838182957884042,Chris Le,1574553637266,5,None,None,None,0x808447fdcd1c3235:0x70372ccdef58e9f,2019,California,...,4.6,1938,₩₩,"[[Tuesday, 11AM–9:30PM], [Wednesday, 11AM–9:30...",{'Accessibility': ['Wheelchair-accessible entr...,Closed ⋅ Opens 11AM,"[0x8084380dd69d9073:0x88a402298a4910d7, 0x8084...",https://www.google.com/maps/place//data=!4m2!3...,None,None


In [6]:
df_maps_reviews.columns.values

array(['user_id', 'name_x', 'time', 'rating', 'text', 'pics', 'resp',
       'gmap_id', 'anio', 'estado', 'name_y', 'address', 'description',
       'latitude', 'longitude', 'category', 'avg_rating',
       'num_of_reviews', 'price', 'hours', 'MISC', 'state',
       'relative_results', 'url', 'state_ab', 'us_state'], dtype=object)

In [7]:
df_maps_reviews.category

0                                        [Korean restaurant]
1                                        [Korean restaurant]
2                                        [Korean restaurant]
3                                        [Korean restaurant]
4                                        [Korean restaurant]
                                 ...                        
2393447    [Coffee shop, Bagel shop, Bakery, Breakfast re...
2393448    [Coffee shop, Bagel shop, Bakery, Breakfast re...
2393449    [Coffee shop, Bagel shop, Bakery, Breakfast re...
2393450    [Coffee shop, Bagel shop, Bakery, Breakfast re...
2393451    [Coffee shop, Bagel shop, Bakery, Breakfast re...
Name: category, Length: 2393452, dtype: object

In [8]:
df_maps_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2393452 entries, 0 to 2393451
Data columns (total 26 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   name_x            object 
 2   time              int64  
 3   rating            int64  
 4   text              object 
 5   pics              object 
 6   resp              object 
 7   gmap_id           object 
 8   anio              int64  
 9   estado            object 
 10  name_y            object 
 11  address           object 
 12  description       object 
 13  latitude          float64
 14  longitude         float64
 15  category          object 
 16  avg_rating        float64
 17  num_of_reviews    int64  
 18  price             object 
 19  hours             object 
 20  MISC              object 
 21  state             object 
 22  relative_results  object 
 23  url               object 
 24  state_ab          object 
 25  us_state          object 
dtypes: float64(3),

In [9]:
df_maps_reviews[~df_maps_reviews.pics.isnull()].sample(3)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,state_ab,us_state
1084735,112092894258987366220,Michael Flores,1560766424975,3,Food was good not great. Pricy for latino food.,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89c2f60419e9336f:0x2b1a38ce6e6c965b,2019,New_York,...,4.3,638,₩₩,"[[Tuesday, 12–11PM], [Wednesday, 12–11PM], [Th...",{'Accessibility': ['Wheelchair-accessible entr...,Open ⋅ Closes 11PM,"[0x89c258580e27e405:0xf0ce7f097d8d2244, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...,None,None
122091,115580060424270600036,Zack Lovatt,1570599223725,5,None,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x80c2c728b96d650b:0xebf33590786b547f,2019,California,...,4.3,235,None,None,"{'Accessibility': None, 'Amenities': ['Bar on ...",None,"[0x80c2c5e852d55701:0xf956891157ada594, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...,None,None
1222563,103056841516918165104,Jack H,1514736212569,4,Perfect drunken pizza stop. Cheap with massiv...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89c259837607afc9:0xfd38fefd373f97a,2017,New_York,...,4.5,66,$,"[[Sunday, 1–11PM], [Monday, 1–11PM], [Tuesday,...","{'Accessibility': None, 'Amenities': ['Good fo...",Closed ⋅ Opens 1PM Mon,"[0x89c2599ede9f6809:0x71a9aa955eb026c2, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...,NY,New York


In [10]:
num_nulls_pics = df_maps_reviews.pics.isnull().sum()
num_nulls_pics

2307136

In [11]:
num_nulls_resp = df_maps_reviews.resp.isnull().sum()
num_nulls_resp

2178044

In [12]:
df_maps_reviews["dtfmt"] = df_maps_reviews.time.apply(
    lambda x: datetime.utcfromtimestamp(x / 1000)
)

df_maps_reviews["mes"] = df_maps_reviews.dtfmt.dt.month

df_maps_reviews["dia"] = df_maps_reviews.dtfmt.dt.day

df_maps_reviews["hora"] = df_maps_reviews.dtfmt.dt.hour

df_maps_reviews.name_x = df_maps_reviews.name_x.str.title()

df_maps_reviews.text = df_maps_reviews.text.str.lower()

df_maps_reviews.sample(3)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,MISC,state,relative_results,url,state_ab,us_state,dtfmt,mes,dia,hora
1994982,100100242701523724095,Phil Iovino,1488696000425,5,had a great lunch. interesting sandwiches and...,None,None,0x88c2e3e4936a3085:0x5d43f1fd0cc206af,2017,Florida,...,{'Accessibility': ['Wheelchair-accessible car ...,Closed ⋅ Opens 10AM,"[0x88c2e1836a77c8db:0x5afbab07e9275d4f, 0x88c2...",https://www.google.com/maps/place//data=!4m2!3...,None,None,2017-03-05 06:40:00.425,3,5,6
161277,113580339616872927649,Veronica Pinon,1562222917291,5,None,None,None,0x808e3280487121eb:0x72ad1dd786b14f33,2019,California,...,{'Accessibility': ['Wheelchair-accessible entr...,Open ⋅ Closes 10PM,"[0x808e2d7e291efb79:0x34dd95929b85da42, 0x808f...",https://www.google.com/maps/place//data=!4m2!3...,None,None,2019-07-04 06:48:37.291,7,4,6
881934,117973005646697209385,Muhammad Ali,1521474371640,5,nice place ... awesome games and delicious foo...,None,None,0x8640c3d0f24bf367:0x9db82341874406a1,2018,Texas,...,"{'Accessibility': None, 'Amenities': ['Good fo...",Permanently closed,"[0x8640c3da744fc37b:0x650c137e7d6f5b63, 0x8640...",https://www.google.com/maps/place//data=!4m2!3...,TX,Texas,2018-03-19 15:46:11.640,3,19,15


#### **_<u>COMENTARIO</u>_**

Las columnas **pics** y **resp** deberían eliminarse porque la cantidades de datos nulos sobrepasa los 90%.

_Obtenemos las columnas de año, mes, día y hora para posteriores análisis_


# Salida Final ETL

In [13]:
df_maps_restaurantes.to_parquet(r'Generated\Google\metada_sitios.parquet')
df_maps_reviews.to_parquet(r'Generated\Google\merge_site_reviews.parquet')
df_yelp_restaurantes.to_parquet(r'Generated\Yelp\bussines.parquet')
df_yelp_checkin.to_parquet(r'Generated\YELP\business_checkin.parquet')
df_yelp_tips.to_parquet(r'Generated\Yelp\business_tip.parquet')
df_yelp_reviews.to_parquet(r'Generated\Yelp\review.parquet')
df_yelp_users.to_parquet(r'Generated\Yelp\users_extracted.parquet')